## Introduction
This notebook is used to convert the prediction file generated by MMYOLO or MMDetection to coco-style pseudo annotations. 

In [ ]:
import os
import json
import pickle
from PIL import Image
import random
import pandas as pd
from tqdm.autonotebook import tqdm
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('dark')

from datetime import datetime as dt
from shapely import Polygon


bboxes_size_px, IoU_thresh = 42.36, 0.5

image_size   = (112, 112)

detection_results_file_path = '../work_dirs/faster-rcnn/LINZ2UGRC/faster-rcnn_train_real_linz_test_syn_linz/prediction.pkl'

## Load Detector Prediction file

RUN ONLY ONCE.

In [ ]:
t_start = dt.now()
print(f'Loading started: {t_start}')
with open(detection_results_file_path, 'rb') as f:
    detection_results = pickle.load(f)
t_end = dt.now()
print(f'Loading ended:   {t_end} | Duration: {t_end-t_start}')

print(f'\nNum. images: {len(detection_results):,d}')

In [ ]:
thresh_conf = 0.6 # confidence threshold for pseudo-labels

## Convert to Pseudo label
Remember to change the saved annotation file name for different stages.

In [ ]:
margin_px = bboxes_size_px/2-1

coco_object_categories = [
    {'id': 1, 'name': 'small'}
]

annotations_coco_RGB = dict(
    categories=coco_object_categories,
    images=[],
    annotations=[]
)


for i_im, detection_results_image in tqdm(enumerate(detection_results), total=len(detection_results)):
    image_file_path = detection_results_image['img_path']
    image_file_name = os.path.basename(image_file_path)
    image = Image.open(image_file_path)
    assert image.size == image_size
    
    annotations_coco_RGB['images'].append(
        {
            'id': i_im,
            'file_name': image_file_name,
            'width': image.size[0],
            'height': image.size[1]
        }
    )
    
    pred_instances = detection_results_image['pred_instances']
    assert (pred_instances['labels'] == 0).all()
    
    mask_instances_toExport = pred_instances['scores'] >= thresh_conf
    
    scores = np.array(pred_instances['scores'][mask_instances_toExport])
    labels = np.array(pred_instances['labels'][mask_instances_toExport])
    bboxes = np.array(pred_instances['bboxes'][mask_instances_toExport,:])


    for (l, t, r, b) in bboxes:
        x_c_bbox = (l+r)/2
        y_c_bbox = (t+b)/2
        
        #####
        bbox_v_trimmed_edge = None
        bbox_h_trimmed_edge = None
        
        if x_c_bbox < margin_px:
            bbox_v_trimmed_edge = 'left'
        
        elif x_c_bbox > image.size[0] - margin_px:
            bbox_v_trimmed_edge = 'right'
    
        if y_c_bbox < margin_px:
            bbox_h_trimmed_edge = 'top'
        
        elif y_c_bbox > image.size[1] - margin_px:
            bbox_h_trimmed_edge = 'bottom'
    
        if bbox_v_trimmed_edge == 'left':
            r_full = r
            l_full = r - bboxes_size_px
    
        elif bbox_v_trimmed_edge == 'right':
            l_full = l
            r_full = l + bboxes_size_px
    
        else:
            l_full = l
            r_full = r
    
    
        if bbox_h_trimmed_edge == 'top':
            b_full = b
            t_full = b - bboxes_size_px
    
        elif bbox_h_trimmed_edge == 'bottom':
            t_full = t
            b_full = t + bboxes_size_px
    
        else:
            t_full = t
            b_full = b
    
        x_c_bbox_full = (l_full+r_full)/2
        y_c_bbox_full = (t_full+b_full)/2


        # Produce fake bbox annotations
        l_RGB = max(0, x_c_bbox_full-bboxes_size_px/2)
        t_RGB = max(0, y_c_bbox_full-bboxes_size_px/2)
        r_RGB = min(x_c_bbox_full+bboxes_size_px/2, image_size[0]-1)
        
        b_RGB = min(y_c_bbox_full+bboxes_size_px/2, image_size[1]-1)
        w_bbox_RGB = r_RGB - l_RGB
        h_bbox_RGB = b_RGB - t_RGB

        annotations_coco_RGB['annotations'].append(
            {
                'iscrowd': 0,
                'category_id': coco_object_categories[0]['id'],
                'id': len(annotations_coco_RGB['annotations']), 
                'image_id': i_im, 
                'bbox': [l_RGB, t_RGB, w_bbox_RGB, h_bbox_RGB],
                'area': w_bbox_RGB*h_bbox_RGB,
                # 'segmentation': []
            }
        )

In [ ]:
image_folder_paths = list(set(map(lambda x: os.path.dirname(x['img_path']), detection_results)))
assert len(image_folder_paths)==1

images_folder_path = image_folder_paths[0]
data_folder_path = os.path.dirname(images_folder_path)
data_folder_path

annotations_coco_RGB_file_path = os.path.join(
    data_folder_path,
    f'annotations_coco_FakeBBoxes:{bboxes_size_px:.2f}px_ForIoU:{IoU_thresh:.3f}_Pseudo-FasterRCNN-SynLINZ-STACKDAAMHeatMaps-ConfThresh:{thresh_conf:.2f}.json'
)
annotations_coco_RGB_file_path

with open(annotations_coco_RGB_file_path, 'w') as f:
    json.dump(annotations_coco_RGB, f)